# DB主要接口展示

In [1]:
from DB import DB

d:\anaconda3\envs\xintuoyin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LLM_path = "../../../../LLM/llama-3-sqlcoder-8b"
db = DB(sqlcoder_path=LLM_path)

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.52s/it]
d:\anaconda3\envs\xintuoyin\lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
d:\anaconda3\envs\xintuoyin\lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


## 获取整个数据库的表信息 get_database_comments()

In [3]:
print(db.get_database_comments())

attendance_employee (新拓银_人事板块_考勤组人员关联表): [id, 考勤组id, 员工id, 部门id, 加入时间, 移除时间, 移除状态, 创建者, 创建时间, 更新者, 更新时间, 是否删除, 租户编号]

attendance_employee_project_hour (新拓银_人事板块_考勤人员_研发工时记录表): [id, 考勤组id, 人员id, 项目id, 排班id, 研发内容, 记录工时, 年, 月, 日, 开始时间, 结束时间, 创建者, 创建时间, 更新者, 更新时间, 是否删除, 租户编号]

attendance_employee_project_hour_period (新拓银_人事板块_考勤人员_研发工时日志_记录表): [主键ID, 研发工时主表ID, 项目id, 研发内容, 开始时间, 结束时间, 记录工时, 创建者, 创建时间, 更新者, 更新时间, 是否删除, 租户编号]

attendance_group (新拓银_人事板块_考勤组信息表): [id, 企业id, 考勤组名字, 考勤组样式，0 代表固定排班，1 代表周期排班, 加入时间, 使用状态, 考勤排序, 创建者, 创建时间, 更新者, 更新时间, 是否删除, 租户编号]

attendance_group_style (新拓银_人事板块_考勤组类型详情表): [id, 企业id, 考勤组名字, 考勤组样式，0 代表固定排班，1 代表周期排班, 加入时间, 使用状态, 周期名字, 周期天数, 法定节假日是否自动排班, 是否跟随考勤组创建, 考勤排序, 创建者, 创建时间, 更新者, 更新时间, 是否删除, 租户编号]

attendance_leave_overtime_records (新拓银_人事板块_考勤管理_加班/请假): [, 类型（加班OVERTIME；请假LEAVE）, 企业ID, 员工ID, 部门id, 排班id, 加班/请假日期, 开始时间, 结束时间, 加班时长(小时), 加班原因, 状态:1有效,0无效, 创建者, 创建时间, 更新者, 更新时间, 是否删除, 租户编号]

attendance_shift_employee_schedule (新拓银_人事板块_考勤人员日期班次表): [id, 考勤组id, 员工id, 排

## 生成SQL语句 generate_sql()

In [5]:
selected_table_name = ['attendance_employee', 'attendance_employee_project_hour', 'attendance_employee_project_hour_period']
question = "2025年二月总工时最长的人员id是？"
sql_code = db.generate_sql(selected_table_name = selected_table_name, question=question)
print(sql_code)

d:\anaconda3\envs\xintuoyin\lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


SELECT aep.id AS employee_id,
       SUM(aep.record_hour) AS total_hours
FROM attendance_employee_project_hour aep
WHERE aep.year = 2025
  AND aep.month = 2
GROUP BY aep.id
ORDER BY total_hours DESC
LIMIT 1;


## 执行SQL语句 execute_sql()

In [6]:
res = db.execute_sql(sql_code)
print(res)

{'rowcount': 1, 'fields': ['employee_id', 'total_hours'], 'data': [{'employee_id': 3, 'total_hours': Decimal('8.00')}]}


## 已知错误类型及解决方案

### SQL语句存在语法错误时

In [ ]:
wrong_SQL = '''
SELECT aep.id AS attendance_id,
       aep.employee_id,
       aep.project_id,
       aep.shift_id,
       aep.start_time,
       aep.end_time,
       aep.record_hour,
       aep.year,
       aep.month,
       aep.day,
       aep.content,
       aep.creator,
       aep.create_time,
       aep.updater,
       aep.update_time,
       aep.deleted,
       aep.tenant_id
FROM attendance_employee_project_hour aep
WHERE aep.employee_id IN
    (SELECT aep2.employee_id
     FROM attendance_employee_project_hour aep2
     GROUP BY aep2.employee_id
     HAVING COUNT(DISTINCT aep2.project_id) > 1)
ORDER BY aep.employee_id NULLS LAST;'''.strip()
res = db.execute_sql(wrong_SQL)
res

{'error': '(1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'NULLS LAST\' at line 24")'}

In [ ]:
revise_prompt ='''
以下是一个在MySQL上执行的语句：
{wrong_sql}

发生报错：{error}

请你修改给出正确的SQL语句，用<SQL>xxx</SQL>形式给出修改后的SQL语句。
'''.strip()

print(revise_prompt.format(wrong_sql=wrong_SQL, error=res['error']))

以下是一个在MySQL上执行的语句：
SELECT aep.id AS attendance_id,
       aep.employee_id,
       aep.project_id,
       aep.shift_id,
       aep.start_time,
       aep.end_time,
       aep.record_hour,
       aep.year,
       aep.month,
       aep.day,
       aep.content,
       aep.creator,
       aep.create_time,
       aep.updater,
       aep.update_time,
       aep.deleted,
       aep.tenant_id
FROM attendance_employee_project_hour aep
WHERE aep.employee_id IN
    (SELECT aep2.employee_id
     FROM attendance_employee_project_hour aep2
     GROUP BY aep2.employee_id
     HAVING COUNT(DISTINCT aep2.project_id) > 1)
ORDER BY aep.employee_id NULLS LAST;

发生报错：(1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'NULLS LAST' at line 24")

请你修改给出正确的SQL语句，用<SQL>xxx<\SQL>形式给出修改后的SQL语句。


通过chatgpt针对该prompt的回复

In [18]:
response='''
MySQL 不支持 `NULLS LAST` 语法，这是 PostgreSQL 的语法。在 MySQL 中，如果你想让 `NULL` 出现在最后，可以使用 `IS NULL` 条件配合排序逻辑进行替代。

你可以将 `ORDER BY aep.employee_id NULLS LAST` 替换为如下等效形式：

```sql
ORDER BY aep.employee_id IS NULL, aep.employee_id
```

这会先按 `aep.employee_id IS NULL` 排序（`FALSE` 在前，`TRUE` 在后），再按 `aep.employee_id` 本身排序。

修改后的 SQL 如下所示：

<SQL>
SELECT aep.id AS attendance_id,  
       aep.employee_id,  
       aep.project_id,  
       aep.shift_id,  
       aep.start_time,  
       aep.end_time,  
       aep.record_hour,  
       aep.year,  
       aep.month,  
       aep.day,  
       aep.content,  
       aep.creator,  
       aep.create_time,  
       aep.updater,  
       aep.update_time,  
       aep.deleted,  
       aep.tenant_id  
FROM attendance_employee_project_hour aep  
WHERE aep.employee_id IN  
    (SELECT aep2.employee_id  
     FROM attendance_employee_project_hour aep2  
     GROUP BY aep2.employee_id  
     HAVING COUNT(DISTINCT aep2.project_id) > 1)  
ORDER BY aep.employee_id IS NULL, aep.employee_id;
</SQL>

是否还需要按其他字段排序或进一步筛选？
'''.strip()

In [20]:
import re

def extract_sql(text):
    # 使用正则表达式提取 <SQL> 和 </SQL> 之间的内容
    match = re.search(r"<SQL>(.*?)</SQL>", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None
sql_code = extract_sql(response).strip()
print(sql_code)

SELECT aep.id AS attendance_id,  
       aep.employee_id,  
       aep.project_id,  
       aep.shift_id,  
       aep.start_time,  
       aep.end_time,  
       aep.record_hour,  
       aep.year,  
       aep.month,  
       aep.day,  
       aep.content,  
       aep.creator,  
       aep.create_time,  
       aep.updater,  
       aep.update_time,  
       aep.deleted,  
       aep.tenant_id  
FROM attendance_employee_project_hour aep  
WHERE aep.employee_id IN  
    (SELECT aep2.employee_id  
     FROM attendance_employee_project_hour aep2  
     GROUP BY aep2.employee_id  
     HAVING COUNT(DISTINCT aep2.project_id) > 1)  
ORDER BY aep.employee_id IS NULL, aep.employee_id;


In [ ]:
res = db.execute_sql(sql_code)
res

{'rowcount': 0,
 'fields': ['attendance_id',
  'employee_id',
  'project_id',
  'shift_id',
  'start_time',
  'end_time',
  'record_hour',
  'year',
  'month',
  'day',
  'content',
  'creator',
  'create_time',
  'updater',
  'update_time',
  'deleted',
  'tenant_id'],
 'data': []}

### 连接超时

In [21]:
res = db.execute_sql(sql_code)
res

{'error': "(2013, 'Lost connection to MySQL server during query ([WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。)')"}

- 重连调用如下方法
- 等待是否有用尚不清楚

In [ ]:
db._init_mysql()